In [1]:
import pandas as pd

# Input requirements and capacities
requirements = [85, 60, 40, 105, 120, 75]
capacities = [100, 100, 100, 100, 100, 100]
setup_cost = 100  # Cost for setting up production in any period
holding_cost = 0.5  # Cost per unit of inventory held at the end of the period

n = len(requirements)
adjusted_requirements = requirements.copy()
production = [0] * n
inventory = [0] * n
total_setup_costs = 0
total_holding_costs = 0

# Back-shift demand where necessary
for i in range(n):
    if adjusted_requirements[i] > capacities[i]:
        excess_demand = adjusted_requirements[i] - capacities[i]
        adjusted_requirements[i] = capacities[i]
        # Distribute excess demand backwards to periods with available capacity
        for j in range(i-1, -1, -1):
            available_capacity = capacities[j] - adjusted_requirements[j]
            if available_capacity >= excess_demand:
                adjusted_requirements[j] += excess_demand
                excess_demand = 0
                break
            else:
                adjusted_requirements[j] += available_capacity
                excess_demand -= available_capacity

# Calculate production and inventory based on adjusted requirements
for i in range(n):
    production[i] = min(adjusted_requirements[i], capacities[i])
    if production[i] > 0:
        total_setup_costs += setup_cost
    if i == 0:
        inventory[i] = production[i] - requirements[i]
    else:
        inventory[i] = inventory[i-1] + production[i] - requirements[i]
    inventory[i] = max(0, inventory[i])
    total_holding_costs += inventory[i] * holding_cost

total_cost = total_setup_costs + total_holding_costs

# Prepare data for DataFrame
data = {
    'Period': range(1, n+1),
    'Requirements': requirements,
    'Proposed Production': production,
    'Ending Inventory': inventory
}

# Create DataFrame
df = pd.DataFrame(data)

# Display DataFrame
print(df.to_string(index=False))

print(f"\nTotal Setup Costs: ${total_setup_costs:.2f}")
print(f"Total Holding Costs: ${total_holding_costs:.2f}")
print(f"Total Cost: ${total_cost:.2f}")


 Period  Requirements  Proposed Production  Ending Inventory
      1            85                   85                 0
      2            60                   60                 0
      3            40                   65                25
      4           105                  100                20
      5           120                  100                 0
      6            75                   75                 0

Total Setup Costs: $600.00
Total Holding Costs: $22.50
Total Cost: $622.50
